In [1]:
import numpy as np
import torch
from torch.utils.data import DataLoader
import os
import cv2
import math

import matplotlib.pyplot as plt
from tqdm import tqdm


print(torch.__version__)
print(torch.device("cuda" if torch.cuda.is_available() else 'cpu'))


c:\Users\user\anaconda3\envs\AGV\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1.13.1+cu116
cuda


raw_data_normal, raw_data_ng에서 1초마다 이미지 프레임 추출 

In [ ]:
video_path='./raw_data_normal'
normal_frame_list = []
#deadlock_frame_list = []
for i,video_file in enumerate(tqdm(os.listdir(video_path))):
    file_name = os.path.splitext(video_file)[0]
   

    video = os.path.join(video_path, video_file)

    Video_Caption = cv2.VideoCapture(video)
    Frame_Rate = Video_Caption.get(5)

    frame_count = 0
    saved_frame_count = 0

    while Video_Caption.isOpened():
        Current_Frame_ID = Video_Caption.get(1)
        ret,frame = Video_Caption.read()
        if not ret:
            break

        if Current_Frame_ID%math.floor(Frame_Rate)==0:
            #gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            Frame_Resize = cv2.resize(frame,(416,416))
            normal_frame_list.append(Frame_Resize)
            saved_frame_count += 1
        frame_count += 1
        
    Video_Caption.release()
print("모든 비디오 파일 처리 완료.")

+ 흑백이미지가 포함되어있는 경우 제외


+ 학습에 사용할 데드락 첫번째 영상에서 AGV가 주행하는 이미지 시퀀스 데이터 제거

+ 이미지 시퀀스를 구성할때 다른 영상으로 넘어가게 되면 사용하지 않고 제거

In [ ]:

#인덱스 번호

#데드락
#######################################################################################
#18, 19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47
#48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76, 77~91, 92~106, 107~121
#122~136, 137~151, 152~166, 167~181, 182~ 196 ,197~211,212~225
############################################################################################
#324~338,339~353,354~368, 369~383, 384~398, 399~413, 414~428, 429~443, 444~458,459~473, 474~488, 489~493, 
# 494~508, 509~523,524~538, 539~554, 555~569, 570, 571,572, 573,574, 575, 576, 577
############################################################################################
#666~680, 681~695, 696~710, 711~725,726~740, 741~755, 756~770, 771~785, 786~ 800, 801~815,816~830, 831~845, 846~860,
#861~875, 876~890, 891~905, 906




#############################################노말
#1~15 16~30,31~45, 46~60,61~75, 76~90, 91~105, 106~120,121~135,136~150,151~165, 166~180, 181~195, 196~210,211~225
#226~240, 241~255. 256~260, 261~275, 276~280, 281~293,
# 
# 354~368, 369~383, 384~398, 399~413, 414~428, 429~443, 444~458
#459~463 464~475, 476~490, 491~505, 506~520, 521~535, 536~550,551~565, 561~575, 576~590, 591

영상마다 이미지 추출하여 Train,Test데이터로 구성

In [ ]:
#normal
temp=[]
#i_1=[x for x in range(1,293+1)]
i_2=[x for x in range(354,591+1)] #테스트

i_list=[i_2]
for i_value in i_list:
    
    for j in i_value:
       
        temp.append(normal_frame_list[j:j+60])
        #for i,t_image in enumerate(temp):
        # plt.imshow(normal_frame_list[j])
        # plt.savefig('./view/'+str(j-354)+'.png')

normal_frame_final=np.array(temp)
normal_labels=np.zeros((normal_frame_final.shape[0],1))
normal_labels = normal_labels.flatten()
normal_labels = normal_labels.astype(int)
np.save('./normal_image_valid.npy',normal_frame_final)
np.save('./normal_labels_valid.npy',normal_labels)

#deadlock
temp=[]
#i_1=[x for x in range(18,225+1)]

#i_2=[x for x in range(666,906+1)]

i_3=[x for x in range(324,577+1)] #테스트
i_list=[i_3]
for i_value in i_list:
    
    for j in i_value:
        temp.append(deadlock_frame_list[j:j+60])
deadlock_frame_final=np.array(temp)


deadlock_labels=np.ones((deadlock_frame_final.shape[0],1))
deadlock_labels = deadlock_labels.flatten()
deadlock_labels = deadlock_labels.astype(int)
np.save('./deadlock_image_valid.npy',deadlock_frame_final)
np.save('./deadlock_labels_valid.npy',deadlock_labels)



데이터셋 저장후 concat

In [ ]:
dataset=np.concatenate((normal_frame_final,deadlock_frame_final),axis=0)
dataset_label=np.concatenate((normal_labels,deadlock_labels),axis=0)
np.save('./image_valid.npy',dataset)
np.save('./label_valid.npy',dataset_label)